In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import pandas as pd
import random
import math
import re

class Tache:
    def __init__(self, nom, patient, competences, duree, docteur=None):
        self.nom = nom
        self.patient = patient
        self.competences = competences
        self.duree = duree
        self.docteur = docteur  # Docteur assigné à la tâche

class Docteur:
    def __init__(self, nom, competences):
        self.nom = nom
        self.competences = competences

    def peut_realiser_tache(self, tache):
        return all(comp in self.competences for comp in tache.competences)

class GestionnaireTaches:
    def __init__(self, docteurs):
        self.taches = []
        self.docteurs = docteurs

    def charger_depuis_csv(self, chemin_fichier):
        data = pd.read_csv(chemin_fichier)
        for _, row in data.iterrows():
            for op in row.index[1:]:
                competences_str = row[op].replace(" et ", ", ").replace("Aucune opération", "")
                competences = competences_str.split(", ") if competences_str else []
                if competences:
                    tache = Tache(op, row.iloc[0], competences, self.calculer_duree(competences_str))
                    self.assigner_docteur(tache)
                    self.taches.append(tache)

    @staticmethod
    def calculer_duree(competences_str):
        return sum(int(m.group(2)) if m.group(2) else 1 for m in re.finditer(r'C\d+(x(\d+))?', competences_str))

    def assigner_docteur(self, tache):
        for docteur in self.docteurs:
            if docteur.peut_realiser_tache(tache):
                tache.docteur = docteur.nom
                break

    def duree_par_patient(self):
        duree_totale_par_patient = {}
        for tache in self.taches:
            if tache.patient in duree_totale_par_patient:
                duree_totale_par_patient[tache.patient] += tache.duree
            else:
                duree_totale_par_patient[tache.patient] = tache.duree
        return duree_totale_par_patient

# Exemple d'utilisation
docteurs = [
    Docteur("Docteur A", ["Compétence A", "Compétence B"]),
    Docteur("Docteur B", ["Compétence C"]),
]

gestionnaire = GestionnaireTaches(docteurs)
gestionnaire.charger_depuis_csv('/kaggle/input/recuit/Fil rouge M2 - Table_competences.csv')  # Remplacez par votre chemin de fichier

duree_par_patient = gestionnaire.duree_par_patient()
print("Durée totale des opérations par patient :")
for patient, duree in duree_par_patient.items():
    print(f"Patient {patient} : {duree} minutes")


Durée totale des opérations par patient :
Patient Patient 1 : 13 minutes
Patient Patient 2 : 5 minutes
Patient Patient 3 : 3 minutes
Patient Patient 4 : 10 minutes
Patient Patient 5 : 8 minutes
Patient Patient 6 : 3 minutes
Patient Patient 7 : 6 minutes
Patient Patient 8 : 7 minutes
Patient Patient 9 : 3 minutes
Patient Patient 10 : 6 minutes


In [7]:
import pandas as pd
import random
import math
import re  # Importation du module re pour les expressions régulières

class Tache:
    def __init__(self, nom, patient, competences, duree):
        self.nom = nom
        self.patient = patient
        self.competences = competences
        self.duree = duree

class GestionnaireTaches:
    def __init__(self):
        self.taches = []

    def charger_depuis_csv(self, chemin_fichier):
        data = pd.read_csv(chemin_fichier)
        for _, row in data.iterrows():
            for op in row.index[1:]:
                competences_str = row[op].replace(" et ", ", ").replace("Aucune opération", "")
                competences = competences_str.split(", ") if competences_str else []
                if competences:
                    # Assurez-vous que l'utilisation de row.iloc[0] est appropriée pour votre cas
                    tache = Tache(op, row.iloc[0], competences, self.calculer_duree(competences_str))
                    self.taches.append(tache)

    @staticmethod
    def calculer_duree(competences_str):
        return sum(int(m.group(2)) if m.group(2) else 1 for m in re.finditer(r'C\d+(x(\d+))?', competences_str))

class RecuitSimule:
    def __init__(self, taches):
        self.taches = taches

    def cout_ordre(self, ordre):
        cout_total = 0
        temps_cumule_par_patient = {}
        for tache in ordre:
            cout_total += tache.duree
            temps_cumule_par_patient[tache.patient] = temps_cumule_par_patient.get(tache.patient, 0) + tache.duree
        cout_total += max(temps_cumule_par_patient.values())
        return cout_total

    def generer_ordre_voisin(self, ordre):
        voisin = ordre[:]
        index1, index2 = random.sample(range(len(ordre)), 2)
        voisin[index1], voisin[index2] = voisin[index2], voisin[index1]
        return voisin

    def optimiser(self, T0, alpha, nb_iterations):
        T = T0
        meilleur_ordre = self.taches[:]
        meilleur_cout = self.cout_ordre(meilleur_ordre)
        for _ in range(nb_iterations):
            ordre_voisin = self.generer_ordre_voisin(meilleur_ordre)
            cout_voisin = self.cout_ordre(ordre_voisin)
            delta_cout = cout_voisin - meilleur_cout
            if delta_cout < 0 or random.random() < math.exp(-delta_cout / T):
                meilleur_ordre = ordre_voisin
                meilleur_cout = cout_voisin
            T *= alpha
        return meilleur_ordre, meilleur_cout

# Utilisation des classes
gestionnaire = GestionnaireTaches()
gestionnaire.charger_depuis_csv('/kaggle/input/recuit/Fil rouge M2 - Table_competences.csv')  # Remplacez par le chemin réel de votre fichier CSV

recuit = RecuitSimule(gestionnaire.taches)
T0 = 100.0
alpha = 0.95
nb_iterations = 1000
ordre_optimal, cout_optimal = recuit.optimiser(T0, alpha, nb_iterations)

# Affichage du résultat
print("Ordre optimal des tâches:")
for tache in ordre_optimal:
    print(f"{tache.nom} - Patient {tache.patient}")
print("Coût optimal:", cout_optimal)


Ordre optimal des tâches:
Opération 1 - Patient Patient 8
Opération 1 - Patient Patient 2
Opération 2 - Patient Patient 1
Opération 2 - Patient Patient 8
Opération 3 - Patient Patient 10
Opération 3 - Patient Patient 6
Opération 1 - Patient Patient 10
Opération 2 - Patient Patient 6
Opération 3 - Patient Patient 7
Opération 5 - Patient Patient 1
Opération 2 - Patient Patient 2
Opération 4 - Patient Patient 8
Opération 1 - Patient Patient 6
Opération 5 - Patient Patient 5
Opération 2 - Patient Patient 3
Opération 4 - Patient Patient 1
Opération 3 - Patient Patient 9
Opération 1 - Patient Patient 1
Opération 2 - Patient Patient 7
Opération 3 - Patient Patient 4
Opération 3 - Patient Patient 8
Opération 3 - Patient Patient 2
Opération 1 - Patient Patient 5
Opération 4 - Patient Patient 10
Opération 4 - Patient Patient 4
Opération 2 - Patient Patient 9
Opération 2 - Patient Patient 4
Opération 1 - Patient Patient 4
Opération 2 - Patient Patient 5
Opération 5 - Patient Patient 4
Opération 4

In [14]:
import pandas as pd
import random
import math
import re

class Tache:
    def __init__(self, nom, patient, competences, duree, docteur=None):
        self.nom = nom
        self.patient = patient
        self.competences = competences
        self.duree = duree
        self.docteur = docteur  # Ajout d'un attribut pour le docteur assigné à la tâche

class GestionnaireTaches:
    def __init__(self, docteurs):
        self.taches = []
        self.docteurs = docteurs  # Liste des docteurs disponibles

    def charger_depuis_csv(self, chemin_fichier):
        data = pd.read_csv(chemin_fichier)
        for _, row in data.iterrows():
            for op in row.index[1:]:
                competences_str = row[op].replace(" et ", ", ").replace("Aucune opération", "")
                competences = competences_str.split(", ") if competences_str else []
                if competences:
                    tache = Tache(op, row.iloc[0], competences, self.calculer_duree(competences_str))
                    self.assigner_docteur(tache)  # Assigner un docteur à la tâche
                    self.taches.append(tache)

    def assigner_docteur(self, tache):
        tache.docteur = random.choice(self.docteurs)  # Assignation aléatoire d'un docteur

    @staticmethod
    def calculer_duree(competences_str):
        return sum(int(m.group(2)) if m.group(2) else 1 for m in re.finditer(r'C\d+(x(\d+))?', competences_str))

    def duree_par_patient(self):
        duree_totale_par_patient = {}
        for tache in self.taches:
            if tache.patient not in duree_totale_par_patient:
                duree_totale_par_patient[tache.patient] = 0
            duree_totale_par_patient[tache.patient] += tache.duree
        return duree_totale_par_patient
    
    

# Exemple d'utilisation
docteurs = ["Dr. Smith", "Dr. Johnson", "Dr. Williams"]  # Liste fictive de docteurs
gestionnaire = GestionnaireTaches(docteurs)
gestionnaire.charger_depuis_csv('/kaggle/input/recuit/Fil rouge M2 - Table_competences.csv')  # Ajustez le chemin selon vos besoins

# Calculer la durée totale des tâches par patient
duree_totale_par_patient = gestionnaire.duree_par_patient()

import pandas as pd
import random
import math
import re

class Tache:
    def __init__(self, nom, patient, competences, duree, docteur=None):
        self.nom = nom
        self.patient = patient
        self.competences = competences
        self.duree = duree
        self.docteur = docteur

class GestionnaireTaches:
    def __init__(self, docteurs):
        self.taches = []
        self.docteurs = docteurs

    def charger_depuis_csv(self, chemin_fichier):
        data = pd.read_csv(chemin_fichier)
        for _, row in data.iterrows():
            for op in row.index[1:]:
                competences_str = row[op].replace(" et ", ", ").replace("Aucune opération", "")
                competences = competences_str.split(", ") if competences_str else []
                if competences:
                    tache = Tache(op, row.iloc[0], competences, self.calculer_duree(competences_str))
                    self.assigner_docteur(tache)
                    self.taches.append(tache)

    def assigner_docteur(self, tache):
        tache.docteur = random.choice(self.docteurs)

    @staticmethod
    def calculer_duree(competences_str):
        return sum(int(m.group(2)) if m.group(2) else 1 for m in re.finditer(r'C\d+(x(\d+))?', competences_str))

    def duree_par_patient(self):
        duree_totale_par_patient = {}
        for tache in self.taches:
            duree_totale_par_patient.setdefault(tache.patient, 0)
            duree_totale_par_patient[tache.patient] += tache.duree
        return duree_totale_par_patient

class RecuitSimule:
    def __init__(self, taches):
        self.taches = taches

    def cout_ordre(self, ordre):
        cout_total = 0
        temps_cumule_par_patient = {}
        for tache in ordre:
            cout_total += tache.duree
            temps_cumule_par_patient.setdefault(tache.patient, 0)
            temps_cumule_par_patient[tache.patient] += tache.duree
        cout_total += max(temps_cumule_par_patient.values()) - min(temps_cumule_par_patient.values())
        return cout_total

    def generer_ordre_voisin(self, ordre):
        voisin = ordre[:]
        index1, index2 = random.sample(range(len(ordre)), 2)
        voisin[index1], voisin[index2] = voisin[index2], voisin[index1]
        return voisin

    def optimiser(self, T0, alpha, nb_iterations):
        T = T0
        meilleur_ordre = self.taches[:]
        meilleur_cout = self.cout_ordre(meilleur_ordre)
        for _ in range(nb_iterations):
            ordre_voisin = self.generer_ordre_voisin(meilleur_ordre)
            cout_voisin = self.cout_ordre(ordre_voisin)
            delta_cout = cout_voisin - meilleur_cout
            if delta_cout < 0 or random.random() < math.exp(-delta_cout / T):
                meilleur_ordre = ordre_voisin
                meilleur_cout = cout_voisin
            T *= alpha
        return meilleur_ordre, meilleur_cout

# Exemple d'utilisation
docteurs = ["Dr. Smith", "Dr. Johnson", "Dr. Williams"]
gestionnaire = GestionnaireTaches(docteurs)
gestionnaire.charger_depuis_csv('/kaggle/input/recuit/Fil rouge M2 - Table_competences.csv')

recuit = RecuitSimule(gestionnaire.taches)
T0 = 100.0
alpha = 0.95
nb_iterations = 1000
ordre_optimal, cout_optimal = recuit.optimiser(T0, alpha, nb_iterations)

print("Coût optimal:", cout_optimal)


# Affichage des résultats
print("Durée totale des tâches par patient :")
for patient, duree in duree_totale_par_patient.items():
    print(f"Patient {patient} : {duree} minutes")


Coût optimal: 74
Durée totale des tâches par patient :
Patient Patient 1 : 13 minutes
Patient Patient 2 : 5 minutes
Patient Patient 3 : 3 minutes
Patient Patient 4 : 10 minutes
Patient Patient 5 : 8 minutes
Patient Patient 6 : 3 minutes
Patient Patient 7 : 6 minutes
Patient Patient 8 : 7 minutes
Patient Patient 9 : 3 minutes
Patient Patient 10 : 6 minutes


Coût optimal: 74
